In [16]:
import os
from dotenv import load_dotenv
from rag_setup import setup_router_query_engine, setup_vector_store, setup_kdbai_session, setup_groq_llm
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core.indices import VectorStoreIndex
from llama_index.core import Document, StorageContext, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

load_dotenv()

True

In [17]:
# Define KDBAI Database and Table Names
DB_NAME = "test_srilanka_rag_database"
TABLE_NAME_CIRCULAR = "test_circular_baseline"
TABLE_NAME_REGULATION = "test_regulation_baseline"
TABLE_NAME_GUIDELINE = "test_guideline_baseline"

# Define HuggingFace Model and Groq LLM Names
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en-v1.5"
GROQ_LLM_NAME = "llama3-groq-70b-8192-tool-use-preview"

In [18]:
# Set up KDBAI session
kdbai_session = setup_kdbai_session()

db = kdbai_session.database('test_srilanka_rag_database')

2024-12-23 10:03:49 - DEBUG : Starting new HTTPS connection (1): cloud.kdb.ai:443
2024-12-23 10:03:49 - DEBUG : https://cloud.kdb.ai:443 "GET /instance/bizxlezc4n/api/v2/ready HTTP/11" 200 2
2024-12-23 10:03:49 - DEBUG : Starting new HTTPS connection (1): cloud.kdb.ai:443
2024-12-23 10:03:49 - DEBUG : https://cloud.kdb.ai:443 "GET /instance/bizxlezc4n/api/v2/version HTTP/11" 200 80
2024-12-23 10:03:49 - INFO : KDBAI session established.
2024-12-23 10:03:49 - DEBUG : Starting new HTTPS connection (1): cloud.kdb.ai:443
2024-12-23 10:03:50 - DEBUG : https://cloud.kdb.ai:443 "GET /instance/bizxlezc4n/api/v2/databases/test_srilanka_rag_database HTTP/11" 200 2849


In [21]:
circular_table = db.table(TABLE_NAME_CIRCULAR)
regulation_table = db.table(TABLE_NAME_REGULATION)
guideline_table = db.table(TABLE_NAME_GUIDELINE)

2024-12-23 10:09:11 - DEBUG : Starting new HTTPS connection (1): cloud.kdb.ai:443
2024-12-23 10:09:12 - DEBUG : https://cloud.kdb.ai:443 "GET /instance/bizxlezc4n/api/v2/databases/test_srilanka_rag_database/tables/test_circular_baseline HTTP/11" 200 935
2024-12-23 10:09:12 - DEBUG : Starting new HTTPS connection (1): cloud.kdb.ai:443
2024-12-23 10:09:12 - DEBUG : https://cloud.kdb.ai:443 "GET /instance/bizxlezc4n/api/v2/databases/test_srilanka_rag_database/tables/test_regulation_baseline HTTP/11" 200 943
2024-12-23 10:09:12 - DEBUG : Starting new HTTPS connection (1): cloud.kdb.ai:443
2024-12-23 10:09:12 - DEBUG : https://cloud.kdb.ai:443 "GET /instance/bizxlezc4n/api/v2/databases/test_srilanka_rag_database/tables/test_guideline_baseline HTTP/11" 200 938


In [23]:
#set up vector store

embedding_model = HuggingFaceEmbedding(
            model_name=EMBEDDING_MODEL_NAME,
            cache_folder=os.getcwd()+'/models'
            )

llm = setup_groq_llm(GROQ_LLM_NAME)

Settings.llm = llm
Settings.embed_model = embedding_model

circular_vector_store = KDBAIVectorStore(circular_table)
circular_index = VectorStoreIndex.from_vector_store(vector_store=circular_vector_store)

guideline_vector_store = KDBAIVectorStore(guideline_table)
guideline_index = VectorStoreIndex.from_vector_store(vector_store=guideline_vector_store)

regulation_vector_store = KDBAIVectorStore(regulation_table)
regulation_index = VectorStoreIndex.from_vector_store(vector_store=regulation_vector_store)
indexes = {
            'guideline': guideline_index,
            'regulation': regulation_index,
            'circular': circular_index
        }


# Set up query engine
query_engine = setup_router_query_engine(indexes, llm)

2024-12-23 10:09:22 - INFO : Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/modules.json HTTP/11" 200 0
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/README.md HTTP/11" 200 0
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/modules.json HTTP/11" 200 0
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/sentence_bert_config.json HTTP/11" 200 0
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/adapter_config.json HTTP/11" 404 0
2024-12-23 10:09:23 - DEBUG : https://huggingface.co:443 "HEAD /BAAI/bge-small-en-v1.5/resolve/main/confi

In [24]:
query_engine.query('What is tea shade?')

2024-12-23 10:09:41 - DEBUG : Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': "Some choices are given below. It is provided in a numbered list (1 to 3), where each item in the list corresponds to a summary.\n---------------------\n(1) Provide information about Sri Lanka Guidelines documents.\n\n(2) Provide information about Sri Lanka Regulations documents and Acts.\n\n(3) Provide information about Sri Lanka Circular documents.\n---------------------\nUsing only the choices above and not prior knowledge, return the top choice(s) (no more than 3, but only select what is needed) by generating the selection object and reasons that are most relevant to the question: 'What is tea shade?'\n"}], 'model': 'llama3-groq-70b-8192-tool-use-preview', 'stream': False, 'temperature': 0.0, 'tool_choice': {'type': 'function', 'function': {'name': 'MultiSelection'}}, 'tools': [{'type': 'function', 'function': {'name': '

Response(response='The concept of tea shade is crucial for tea plantations, especially in areas with intense sunlight. It involves planting trees in a way that provides the right amount of shade for the tea plants, which is important for their growth. The kind and number of shade trees can really change how well the tea grows and how much of it you can harvest.', source_nodes=[NodeWithScore(node=TextNode(id_='661f19fc-c9dd-40da-b434-44bc68b4772e', embedding=None, metadata={'__nn_distance': 0.7458294630050659, 'issue_date': 'March 2003', 'issuing_authority': 'Tea Research Institute', 'class': 'circular', 'reference_number': 'SI 925', 'title': 'ADVVISORY CIRCULAR No.SI 925 SHADING IN TEA', 'original_doc_id': 70}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='# 2.2\r\n\r\nIn mature tea, where there is no shade, the best time to plant shade trees is after plucking; the branches of tea bus